# Laboratório Exame
## Equipe:
## Pedro Anacleto Martins Senna De Oliveira
## André Luiz de Melo Thissen
## Nikollas da Silva Antes
## Pedro Luchiari de Carvalho

In [22]:
import pandas as pd
import numpy as np

In [23]:
teste = pd.read_csv('exame_cmc13_dados_teste.csv',sep=';',on_bad_lines='skip')
treinamento = pd.read_csv('exame_cmc13_dados_treinamento.csv',sep=';',on_bad_lines='skip')

treinamento

,Unnamed: 0.1,Unnamed: 0,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_l,Language,Category,city,state,country
0,98,98,9,34.7439,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],germantown,tennessee,usa
1,99,99,243,34.7439,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],arden hills,minnesota,usa
2,100,100,388,35.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],nashville,tennessee,usa
3,103,103,3363,29.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],knoxville,tennessee,usa
4,105,105,6795,31.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],portland,oregon,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131174,511368,511368,247055,25.0000,067169507X,0,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",daly city,california,usa
131175,511369,511369,253696,62.0000,067169507X,10,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",waldport,oregon,usa
131176,511370,511370,254971,53.0000,067169507X,8,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",boise,idaho,usa
131177,511371,511371,267326,37.0000,067169507X,0,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",longwood,florida,usa


In [24]:
print(treinamento.isnull().values.any())
print(teste.isnull().values.any())
treinamento = treinamento.dropna(how='any')
print(treinamento.isnull().values.any())
treinamento

True
True
False


,Unnamed: 0.1,Unnamed: 0,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_l,Language,Category,city,state,country
0,98,98,9,34.7439,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],germantown,tennessee,usa
1,99,99,243,34.7439,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],arden hills,minnesota,usa
2,100,100,388,35.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],nashville,tennessee,usa
3,103,103,3363,29.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],knoxville,tennessee,usa
4,105,105,6795,31.0000,0440234743,0,The Testament,John Grisham,1999.0,Dell,http://images.amazon.com/images/P/0440234743.0...,en,['Fiction'],portland,oregon,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131174,511368,511368,247055,25.0000,067169507X,0,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",daly city,california,usa
131175,511369,511369,253696,62.0000,067169507X,10,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",waldport,oregon,usa
131176,511370,511370,254971,53.0000,067169507X,8,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",boise,idaho,usa
131177,511371,511371,267326,37.0000,067169507X,0,SEAT OF THE SOUL,Gary Zukav,1990.0,Free Press,http://images.amazon.com/images/P/067169507X.0...,en,"['Body, Mind & Spirit']",longwood,florida,usa
